In [1]:
import sys
!{sys.executable} -m pip install liac-arff



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11768 sha256=17f8751ac9d0d1810bac7324dd3afe4ba4077dcbf2b45e282131bc4852398fd3
  Stored in directory: /Users/emmabachura/Library/Caches/pip/wheels/93/f3/5b/658a9bddee916a5f4b84bdc1a4e0fabd22fb17947c2c9542e6
Successfully built liac-arff


In [2]:
import sys
!{sys.executable} -m pip install pandas numpy liac-arff scikit-learn matplotlib seaborn




In [3]:
import pandas as pd
import numpy as np
import arff
from pathlib import Path

print("All imports are working!")


All imports are working!


In [4]:

from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import joblib


In [6]:
import arff
from pathlib import Path
import pandas as pd

path = Path("../data/traffic-violations.csv")  # even if extension says .csv
with open(path, "r") as f:
    raw = arff.load(f)

df = pd.DataFrame(raw["data"], columns=[a[0] for a in raw["attributes"]])
df.head(), df.shape


(                                         Description Belts Personal.Injury  \
 0  DISPLAYING EXPIRED REGISTRATION PLATE ISSUED B...    No              No   
 1  DRIVER FAIL TO STOP AT RED TRAFFIC SIGNAL BEFO...    No              No   
 2      DRIVING UNDER THE INFLUENCE OF ALCOHOL PER SE    No              No   
 3  PERSON DRIVING MOTOR VEHICLE ON HIGHWAY OR PUB...    No              No   
 4  DISPLAYING EXPIRED REGISTRATION PLATE ISSUED B...    No              No   
 
   Property.Damage Commercial.License Commercial.Vehicle State  \
 0              No                 No                 No    NC   
 1              No                 No                 No    MD   
 2              No                 No                 No    MD   
 3              No                 No                 No    MD   
 4              No                Yes                 No    MD   
 
        VehicleType    Year     Make  ...   Color   Charge  \
 0  02 - Automobile  2013.0  HYUNDAI  ...    GRAY   13411f   
 1

In [7]:
df.to_csv("../data/datasetone_fixed.csv", index=False)


In [8]:
df = pd.read_csv("../data/datasetone_fixed.csv")
df.head()


,Description,Belts,Personal.Injury,Property.Damage,Commercial.License,Commercial.Vehicle,State,VehicleType,Year,Make,...,Color,Charge,Contributed.To.Accident,Race,Gender,Driver.City,Driver.State,DL.State,Arrest.Type,Violation.Type
0,DISPLAYING EXPIRED REGISTRATION PLATE ISSUED B...,No,No,No,No,No,NC,02 - Automobile,2013.0,HYUNDAI,...,GRAY,13411f,No,WHITE,F,ASHEVILLE,NC,NC,A - Marked Patrol,Citation
1,DRIVER FAIL TO STOP AT RED TRAFFIC SIGNAL BEFO...,No,No,No,No,No,MD,02 - Automobile,2015.0,FORD,...,SILVER,21202i1,No,OTHER,M,SILVER SPRING,MD,MD,A - Marked Patrol,Citation
2,DRIVING UNDER THE INFLUENCE OF ALCOHOL PER SE,No,No,No,No,No,MD,02 - Automobile,2000.0,TOYOTA,...,BLACK,21902a2,No,BLACK,M,SILVER SPRING,MD,MD,B - Unmarked Patrol,Citation
3,PERSON DRIVING MOTOR VEHICLE ON HIGHWAY OR PUB...,No,No,No,No,No,MD,02 - Automobile,2012.0,HOND,...,BLACK,16303c,No,BLACK,M,COLUMBIA,MD,MD,A - Marked Patrol,Citation
4,DISPLAYING EXPIRED REGISTRATION PLATE ISSUED B...,No,No,No,Yes,No,MD,02 - Automobile,2010.0,FORD,...,BLACK,13411f,No,WHITE,M,MOUNT AIRY,MD,MD,A - Marked Patrol,Citation


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70340 entries, 0 to 70339
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Description              70340 non-null  object 
 1   Belts                    70340 non-null  object 
 2   Personal.Injury          70340 non-null  object 
 3   Property.Damage          70340 non-null  object 
 4   Commercial.License       70340 non-null  object 
 5   Commercial.Vehicle       70340 non-null  object 
 6   State                    70337 non-null  object 
 7   VehicleType              70340 non-null  object 
 8   Year                     69906 non-null  float64
 9   Make                     69892 non-null  object 
 10  Model                    69885 non-null  object 
 11  Color                    69452 non-null  object 
 12  Charge                   70340 non-null  object 
 13  Contributed.To.Accident  70340 non-null  object 
 14  Race                  

In [10]:
df.describe(include='all')

,Description,Belts,Personal.Injury,Property.Damage,Commercial.License,Commercial.Vehicle,State,VehicleType,Year,Make,...,Color,Charge,Contributed.To.Accident,Race,Gender,Driver.City,Driver.State,DL.State,Arrest.Type,Violation.Type
count,70340,70340,70340,70340,70340,70340,70337,70340,69906.000000,69892,...,69452,70340,70340,70340,70340,70332,70340,70288,70340,70340
unique,2130,2,2,2,2,2,57,22,NaN,888,...,26,605,2,6,3,1889,57,63,19,3
top,DRIVER FAILURE TO OBEY PROPERLY PLACED TRAFFIC...,No,No,No,No,No,MD,02 - Automobile,NaN,TOYOTA,...,BLACK,218011,No,WHITE,M,SILVER SPRING,MD,MD,A - Marked Patrol,Warning
freq,5733,67954,69534,68984,68250,70003,61540,61789,NaN,8036,...,14174,8938,68655,24973,47163,17691,63593,61346,58152,34382
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004.947959,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.359833,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.isna().sum()

Description                  0
Belts                        0
Personal.Injury              0
Property.Damage              0
Commercial.License           0
Commercial.Vehicle           0
State                        3
VehicleType                  0
Year                       434
Make                       448
Model                      455
Color                      888
Charge                       0
Contributed.To.Accident      0
Race                         0
Gender                       0
Driver.City                  8
Driver.State                 0
DL.State                    52
Arrest.Type                  0
Violation.Type               0
dtype: int64

In [12]:
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("Categorical:", cat_cols)
print("Numeric:", num_cols)


Categorical: ['Description', 'Belts', 'Personal.Injury', 'Property.Damage', 'Commercial.License', 'Commercial.Vehicle', 'State', 'VehicleType', 'Make', 'Model', 'Color', 'Charge', 'Contributed.To.Accident', 'Race', 'Gender', 'Driver.City', 'Driver.State', 'DL.State', 'Arrest.Type', 'Violation.Type']
Numeric: ['Year']


In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [14]:
# Columns detected earlier
categorical = cat_cols
numeric = num_cols

In [15]:
# Imputers + encoders
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])


In [ ]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric),
        ("cat", categorical_transformer, categorical)
    ]
)

print("Preprocessor ready!")

Preprocessor ready!


In [17]:
target =  "Violation.Type"
X = df.drop(columns=[target])
y = df[target]
print("X shape:", X.shape)
print("y shape:", y.shape)
y.value_counts()

X shape: (70340, 20)
y shape: (70340,)


Violation.Type
Warning     34382
Citation    32452
SERO         3506
Name: count, dtype: int64

In [19]:
# Recompute feature lists from X (target already dropped)
categorical = X.select_dtypes(include=["object"]).columns.tolist()
numeric     = X.select_dtypes(include=["int64","float64"]).columns.tolist()

# (Re)build the preprocessor using feature columns from X
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric),
    ("cat", categorical_transformer, categorical)
])


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)



Train size: (56272, 20)
Test size: (14068, 20)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

models = {
    "LogReg": LogisticRegression(max_iter=300),
    "RandomForest": RandomForestClassifier(n_estimators=150, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=7)
}

results = {}

for name, model in models.items():
    pipe = Pipeline([("preprocess", preprocessor), ("model", model)])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)

    acc = accuracy_score(y_test, preds)
    results[name] = acc

    print("\n============================")
    print(name)
    print("Accuracy:", acc)
    print(classification_report(y_test, preds))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report


knn_results = []
for k in [3, 5, 11]:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn_pipe = Pipeline([("preprocess", preprocessor), ("model", knn)])
    knn_pipe.fit(X_train, y_train)
    knn_preds = knn_pipe.predict(X_test)

    acc = accuracy_score(y_test, knn_preds)
    f1w = f1_score(y_test, knn_preds, average="weighted")
    knn_results.append((k, acc, f1w))

    print(f"\n=== KNN (k={k}) ===")
    print("Accuracy:", round(acc, 4), " | F1(w):", round(f1w, 4))
    print(classification_report(y_test, knn_preds))


In [ ]:
print("\n=== SUMMARY OF ALL MODELS (DATASET 1) ===")
print(f"LogReg Accuracy:       {results['LogReg']:.4f}")
print(f"RandomForest Accuracy: {results['RandomForest']:.4f}")

for k, acc, f1 in knn_results:
    print(f"KNN (k={k}) Accuracy:   {acc:.4f} | F1: {f1:.4f}")
